<a href="https://colab.research.google.com/github/AlphaSierraDelta2849/Fraud-Detection/blob/main/Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Déploiement modèle:
Nous allons déployé notre modèle pour un suivi en temps réél sur de nouvelles données. Pour cela nous allons utiliser spark streaming

#Configuration environnement

##installation outils nécessaires

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j
!pip install pyspark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,974 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubun

##Importation modules nécessaires

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.ml import PipelineModel
from google.colab import drive
from google.colab import drive

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()


'/usr/local/lib/python3.10/dist-packages/pyspark'

##Instanciation du spark context

In [3]:
sc = SparkContext("local[*]", "ModelDeployment")
ssc = StreamingContext(sc, 5)  # Batch interval of 5 seconds

/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


##Importation modèle:
Nous avons choisi d'importer et déployer le modèle de régression logistique.

In [6]:
drive.mount('/content/drive')
my_path="/content/drive/My Drive/csv_files_colab/fraud_detection/models/lr"
model = PipelineModel.load(my_path)
print(model)

Mounted at /content/drive
PipelineModel_83573dcab30b


##Déploiement

In [7]:

# Define the processing logic for each batch
def process_batch(batch):
    # Your processing logic here
    predictions = model.transform(batch)
    predictions.show()

stream = ssc.socketTextStream("localhost", 9999)

# Process each batch of data
stream.foreachRDD(process_batch)



# Step 6: Start streaming context
ssc.start()
ssc.awaitTermination()

Le déploiement nous permet de suivre notre en temps réél sur de nouvelles données.